In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import train,prepare,evaluate
from models.Encoders import NRMS_Encoder, FIM_Encoder, NPA_Encoder, Pipeline_Encoder, MHA_Encoder, Bert_Encoder
from models.SFI import SFI_unified

In [2]:
name='sfi'
hparams = {
    'select':'unified',
    'integration':'gate',
    'scale':'demo',
    'mode':'train',
    'device':'cuda:1',
    'epochs':1,
    'batch_size':100,
    'title_size':20,
    'his_size':50,
    'k':30,
    'npratio':4,
    'dropout_p':0.2,
    'embedding_dim':300,
    'value_dim':16,
    'query_dim':200,
    'head_num':16,
    'filter_num':150,
    'level':1,
    'metrics':'auc,mean_mrr,ndcg@5,ndcg@10',
    'attrs': ['title'],
    'save_step':[0],
    'train_embedding':True,
}
torch.cuda.set_device(hparams['device'])

In [3]:
vocab, loaders = prepare(hparams)

[2021-03-16 16:39:58,601] INFO (root) Hyper Parameters are
{'select': 'unified', 'integration': 'gate', 'scale': 'demo', 'mode': 'train', 'device': 'cuda:1', 'epochs': 1, 'batch_size': 100, 'title_size': 20, 'his_size': 50, 'k': 30, 'npratio': 4, 'dropout_p': 0.2, 'embedding_dim': 300, 'value_dim': 16, 'query_dim': 200, 'head_num': 16, 'filter_num': 150, 'level': 1, 'metrics': 'auc,mean_mrr,ndcg@5,ndcg@10', 'attrs': ['title'], 'save_step': [0], 'train_embedding': True}
[2021-03-16 16:39:58,603] INFO (root) preparing dataset...
[2021-03-16 16:40:00,860] INFO (torchtext.vocab) Loading vectors from .vector_cache/glove.840B.300d.txt.pt


In [5]:
hparams['select'] = 'unified'
hparams['name'] = '-'.join([name,'fim',hparams['select']])

fim_encoder = FIM_Encoder(hparams, vocab)
sfi = SFI_unified(hparams, fim_encoder).to(hparams['device'])

In [6]:
sfi = train(sfi, hparams, loaders, interval=10)

[2021-03-16 16:40:22,132] INFO (root) training...
epoch 1 , step 30 , loss: 1.6320: : 32it [00:06,  4.58it/s]
[2021-03-16 16:40:29,836] INFO (root) saved model of epoch 1 at data/model_params/sfi-fim-unified/demo_epoch1_step0_[hs=50,topk=30].model
[2021-03-16 16:40:29,843] INFO (root) evaluating...
186it [00:05, 34.20it/s]
[2021-03-16 16:40:36,294] INFO (root) evaluation results:{'auc': 0.5597, 'mean_mrr': 0.2578, 'ndcg@5': 0.2732, 'ndcg@10': 0.3315, 'step': 0}


In [ ]:
hparams['name'] = '-'.join([name,'nrms',hparams['select']])
nrms_encoder = NRMS_Encoder(hparams, vocab)
sfi = SFI_unified(hparams, nrms_encoder).to(hparams['device'])

In [ ]:
sfi = train(sfi, hparams, loaders, interval=10)

In [ ]:
hparams['name'] = '-'.join([name,'npa',hparams['select']])

hparams['user_dim'] = 200
hparams['query_dim'] = 200
hparams['filter_num'] = 400

npa_encoder = NPA_Encoder(hparams, vocab, len(loaders[0].dataset.uid2index))
sfi = SFI_unified(hparams, npa_encoder).to(hparams['device'])

In [ ]:
sfi = train(sfi, hparams, loaders, interval=10)

In [ ]:
hparams['name'] = '-'.join([name,'pipeline',hparams['select']])

hparams['pipeline'] = 'sfi-fim'
# hparams['scale'] = 'large'

ppl_encoder = Pipeline_Encoder(hparams)
sfi = SFI_unified(hparams, ppl_encoder).to(hparams['device'])

In [ ]:
sfi = train(sfi, hparams, loaders, interval=10)

In [ ]:
hparams['name'] = '-'.join([name,'mha',hparams['select']])

mha_encoder = MHA_Encoder(hparams, vocab)
sfi = SFI_unified(hparams, mha_encoder).to(hparams['device'])

In [ ]:
sfi = train(sfi, hparams, loaders, interval=10)

In [ ]:
hparams['name'] = '-'.join([name,'bert',hparams['select']])

hparams['level'] = 2
hparams['batch_size'] = 10
hparams['learning_rate'] = 3e-5

hparams['bert'] = 'bert-base-uncased'
# hparams['bert'] = 'albert-base-v2'

_, loaders = prepare(hparams)

In [ ]:
bert_encoder = Bert_Encoder(hparams)
sfi = SFI_unified(hparams, bert_encoder).to(hparams['device'])

In [ ]:
sfi = train(sfi, hparams, loaders, interval=10)

## The rest is for development

In [ ]:
record = next(iter(loaders[0]))